In [ ]:
###Imports

In [1]:
import pandas as pd
from scipy.interpolate import interp1d
from sklearn.preprocessing import MinMaxScaler

import plotly
import plotly.express as px

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output 

In [ ]:
###read data

In [2]:
data = pd.read_csv("https://firms2.modaps.eosdis.nasa.gov/data/active_fire/noaa-20-viirs-c2/csv/J1_VIIRS_C2_Global_7d.csv")
data.head(20)

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,confidence,version,bright_ti5,frp,daynight
0,59.56768,151.47574,336.2,0.61,0.71,2020-09-26,24,1,nominal,2.0NRT,282.3,10.5,D
1,59.59822,151.52393,339.2,0.61,0.71,2020-09-26,24,1,nominal,2.0NRT,283.5,9.8,D
2,59.60613,151.52197,326.6,0.61,0.71,2020-09-26,24,1,nominal,2.0NRT,282.8,9.4,D
3,59.60234,151.51353,356.4,0.61,0.71,2020-09-26,24,1,low,2.0NRT,285.0,9.8,D
4,59.61081,151.51282,336.4,0.61,0.71,2020-09-26,24,1,nominal,2.0NRT,284.6,15.8,D
5,59.61471,151.52150,356.3,0.61,0.71,2020-09-26,24,1,low,2.0NRT,289.7,15.8,D
6,59.62854,151.51318,333.2,0.61,0.71,2020-09-26,24,1,nominal,2.0NRT,284.7,5.7,D
7,59.63213,151.46237,331.2,0.61,0.71,2020-09-26,24,1,nominal,2.0NRT,285.0,3.8,D
8,59.59597,151.51820,341.9,0.61,0.71,2020-09-26,24,1,nominal,2.0NRT,285.1,13.1,D
9,59.60391,151.51620,356.0,0.61,0.71,2020-09-26,24,1,low,2.0NRT,284.9,15.9,D


In [ ]:
###extract required features
###add intensity feature based on confidence and brightness value
###process intensity

In [3]:
rdata = data[["longitude","latitude","acq_date","confidence","bright_ti5"]]
rdata["intensity"] = 1.0
for i in rdata.index :
    if rdata["confidence"][i]==("high") :
        rdata["intensity"][i] = 100*rdata["bright_ti5"][i]
    elif rdata["confidence"][i]==("nominal") :
        rdata["intensity"][i] = 50*rdata["bright_ti5"][i]
    elif rdata["confidence"][i]==("low") :
        rdata["intensity"][i] = rdata["bright_ti5"][i]
rdata.head()

/home/aaditya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/aaditya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/aaditya/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3331: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/aa

,longitude,latitude,acq_date,confidence,bright_ti5,intensity
0,151.47574,59.56768,2020-09-26,nominal,282.3,14115.0
1,151.52393,59.59822,2020-09-26,nominal,283.5,14175.0
2,151.52197,59.60613,2020-09-26,nominal,282.8,14140.0
3,151.51353,59.60234,2020-09-26,low,285.0,285.0
4,151.51282,59.61081,2020-09-26,nominal,284.6,14230.0


In [4]:
scaler = MinMaxScaler(feature_range=(0,5))
rdata["intensity"] = scaler.fit_transform(rdata[["intensity"]])
rdata["intensity"] = rdata["intensity"].round().astype(int)
rdata.head()

/home/aaditya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/aaditya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,longitude,latitude,acq_date,confidence,bright_ti5,intensity
0,151.47574,59.56768,2020-09-26,nominal,282.3,2
1,151.52393,59.59822,2020-09-26,nominal,283.5,2
2,151.52197,59.60613,2020-09-26,nominal,282.8,2
3,151.51353,59.60234,2020-09-26,low,285.0,0
4,151.51282,59.61081,2020-09-26,nominal,284.6,2


In [ ]:
# trace1 = rdata.loc[rdata["acq_date"]=="2020-09-26"]
# trace1

In [ ]:
###Scatter-geo plot

In [ ]:
fig = px.scatter_geo(rdata.head(10000),lat="latitude",lon="longitude",
                     color="intensity",
                     title="VIIRS Data - Last 7 Days",
                     projection="natural earth"
                     )
fig.update_layout(height=300, margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
###Heat maps

In [ ]:
severity=rdata["intensity"].values.tolist()
interp1dobj = interp1d([0,max(severity)],[1,12])
circle_radius=interp1dobj(severity)
circle_radius

In [ ]:
print("open-street-map")
heat_fig = px.density_mapbox(rdata.head(100000),lat="latitude",lon="longitude",radius=circle_radius,zoom=2,mapbox_style="open-street-map")
heat_fig.show()

In [ ]:
# print("carto-positron")
# heat_fig = px.density_mapbox(rdata.head(100000),lat="latitude",lon="longitude",radius=circle_radius,zoom=2,mapbox_style="carto-positron")
# heat_fig.show()

In [ ]:
# print("carto-darkmatter")
# heat_fig = px.density_mapbox(rdata.head(100000),lat="latitude",lon="longitude",radius=circle_radius,zoom=2,mapbox_style="carto-darkmatter")
# heat_fig.show()

In [ ]:
# print("stamen-terrain")
# heat_fig = px.density_mapbox(rdata.head(100000),lat="latitude",lon="longitude",radius=circle_radius,zoom=2,mapbox_style="stamen-terrain")
# heat_fig.show()

In [ ]:
# print("stamen-toner")
# heat_fig = px.density_mapbox(rdata.head(100000),lat="latitude",lon="longitude",radius=circle_radius,zoom=2,mapbox_style="stamen-toner")
# heat_fig.show()

In [ ]:
# print("stamen-watercolor")
# heat_fig = px.density_mapbox(rdata.head(100000),lat="latitude",lon="longitude",radius=circle_radius,zoom=2,mapbox_style="stamen-watercolor")
# heat_fig.show()

In [ ]:
###Scatter plot with slider

In [5]:
mark_values = {
    1 : "2020-09-26",
    2 : "2020-09-27",
    3 : "2020-09-28",
    4 : "2020-09-29",
    5 : "2020-09-30",
    6 : "2020-10-01",
    7 : "2020-10-02",
    8 : "2020-10-03"
}

In [6]:
app = dash.Dash()
app.layout = html.Div([
    
    html.Div([
        dcc.Graph(id="viirs")
    ]),
    
    html.Div([
        html.Label(["Choose a day to view data"],style={"text-align":"center","font-weight":"bold"}),
    ]),
        
        
    html.Div([
       dcc.Slider(
           id="slider",
           min = 1,
           max = 8,
           value = 1,
           marks = mark_values,
           step = None,
        ), 
    ],style={"width":"70%","position":"absolute","left":"5%"}),
        
])

In [7]:
@app.callback(
    Output('viirs','figure'),
    [Input('slider','value')],
)
def update_graph(day_choice):
    trace = rdata.loc[rdata["acq_date"]==day_choice]
    days_scatter = px.scatter_geo(trace.head(1000),
                                  lat="latitude",
                                  lon="longitude",
                                  color="intensity",
                                  projection="natural earth",
                     )
    days_scatter.update_traces(textposition="top center")
    return(days_scatter)

In [8]:
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
